In [39]:
from typing import List
import geojson
import requests
import pandas as pd
#from osgeo import gdal
import countries_iso_dict
import iso_codes
from requests_cache import CachedSession

_session = CachedSession(expire_after=604800) #cache expires after 1 week

def clear_cache():
    _session.cache.clear()

def set_cache_expire_time(seconds: int):
    """Update cache expiring time. Does not clear cache."""
    global _session
    _session = CachedSession(expire_after=seconds)

def disable_cache():
    global _session
    _session = requests

def _is_valid_adm(iso3, adm: str) -> bool :
    html = _session.get("https://www.geoboundaries.org/api/current/gbOpen/{}/".format(iso3), verify=True).text
    #print('adm in html =' + str(adm in html))
    return adm in html

def _validate_adm(adm: str | int) -> str :
    if type(adm).__name__ == 'int' or len(str(adm)) == 1:
        adm = 'ADM' + str(adm)
    if str.upper(adm) in ['ADM{}'.format(str(i)) for i in range(6)] or str.upper(adm) == 'ALL':
        return str.upper(adm)
    raise KeyError

def _get_smallest_adm(iso3):
    current_adm = 5
    adm_exists = False
    while current_adm >= 0:
        #print('testing adm'+str(current_adm))
        if _is_valid_adm(iso3, 'ADM' + str(current_adm)):
            break
        current_adm -= 1
    print('Smallest ADM level found for {} : ADM{}'.format(iso3, current_adm))
    return 'ADM' + str(current_adm)

def _is_valid_iso3_code(territory: str) -> bool :
    return str.lower(territory) in iso_codes.iso_codes

def _get_iso3_from_name_or_iso2(name: str) -> str:
    try :
        return str.upper(countries_iso_dict.countries_iso3[str.lower(name)])
    except KeyError as e:
        print("KeyError : Couldn't find country named {}".format(e))
        raise KeyError

def _generate_url(territory: str, adm : str | int) -> str :
    iso3 = str.upper(territory) if _is_valid_iso3_code(territory) else _get_iso3_from_name_or_iso2(territory)
    if adm != -1:
        adm = _validate_adm(adm)
    else:
        adm = _get_smallest_adm(iso3)
    if not _is_valid_adm(iso3, adm):
        print("KeyError : ADM level '{}' doesn't exist for country '{}' ({})".format(adm, territory, iso3))
        raise KeyError
    return "https://www.geoboundaries.org/api/current/gbOpen/{}/{}/".format(iso3, adm)

def get_metadata(territory: str, adm: str | int) -> dict:
    """
    Returns a json of specifided territory's metadata.
    Use adm='ALL' to get metadata for every ADM levels.
    """
    return _session.get(_generate_url(territory, adm), verify=True).json() #TO DO get rid of verify arg

def _get_data(territory: str, adm: str, simplified: bool) -> dict:
    """Requests the geoboundaries API and returns a JSON str object of the specified territory and ADM """
    geom_complexity = 'simplifiedGeometryGeoJSON' if simplified else 'gjDownloadURL'
    try:
        json_uri = get_metadata(territory, adm)[geom_complexity]
    except:
        print("Error while requesting geoboudaries API\n URL : {}\n".format(_generate_url(territory, adm)))
        raise
    return _session.get(json_uri).text

def get_adm(territories: str | List[str], adm: str | int, simplified=True) -> dict:
    """
    Returns a json of specifided territories at specifided adm levels.

    Allowed values for <territories> argument :

        - ISO 3166-1 (alpha2) : AFG, QAT, YEM, etc.
        - ISO 3166-1 (alpha3) : AF, QA, YE, etc.
        - Country name (i.e ADM0 territories) in any of the following languages : Arabic, Armenian, Basque, Bulgarian, Chinese (simplified), Chinese (traditional), Czech,
             Danish, Dutch, English, Esperanto, Estonian, Finnish, French, German, Greek, Hungarian, Italian, Japanese, Korean, Lithuanian,
             Norwegian, Polish, Portuguese, Romanian, Russian, Slovak, Spanish, Swedish, Thai, Ukrainian
        For more information, check out https://stefangabos.github.io/world_countries/ (the data source for ISO codes and countries' names)

    Allowed format for <territories> argument :

        - a single string : "Senegal", "SEN", "เซเนกัล"
        - a list of strings : ["SEN", "Mali'], ["セネガル", "մալի"]

    Allowed values for <adm> argument :
        - 'ADM0' to 'ADM5' (if exists for specified country)
        - int 0 to 5
        - int -1 (returns the smallest available ADM level)
        For more information about ADM levels, check out https://www.geoboundaries.org/index.html
    """

    if type(territories) == str:
        return geojson.loads(_get_data(territories, adm, simplified))
    geojsons = [geojson.loads(_get_data(i, adm, simplified))['features'][0] for i in territories]
    feature_collection = geojson.FeatureCollection(geojsons)
    return feature_collection

In [46]:
regions_dict = {
    "South Sudan": [
        "Unity", "Jonglei", "Western Bahr El Ghazal", "Abyei Administrative Area", "Renk", "Geziret Aba Village", "Rabak Locality", 
        "White Nile", "Khartoum", "South Darfur", "North Darfur", "Bentiu", "Malakal", "Juba", "Tonj North", "Greater Upper Nile", 
        "Aweil East", "Aweil South", "Central Equatoria", "Greater Pibor Administrative Area", "Ruweng", "Bor", "Pochalla", 
        "Rumbek East", "Yirol West", "Yirol East", "Lakes", "Bor Capital", "Northern Bahr El Ghazal", "Western Equatoria", 
        "National South Sudan-Wide", "Greater Darfur", "Greater Kordofan", "Red Sea", "El Obeid North Kordofan", "El Fao El Gedaref", 
        "El Fasher", "Nasir", "Mayom", "Panyijiar", "Koch", "Kajo-Keji", "Maban", "Old Fangak", "Sudd Marshes", "Sudd Wetlands", 
        "Wau", "Lakes State", "Upper Nile", "Greater Bahr El Ghazal", "Gogrial West", "Awerial", "Gak", "pibor"
    ],
    "Sudan": [
        "Khartoum", "South Darfur", "North Darfur", "El Obeid North Kordofan", "El Fao El Gedaref", "Greater Darfur", "Greater Kordofan", 
        "Red Sea", "Kassala", "River Nile", "Aj Jazira", "Sennar", "North Kordofan", "Merwoe", "Gezira", "East Darfur", "White Nile", 
        "Blue Nile", "Kosti", "Sudan Nile Basin"
    ],
    "Ethiopia": [
        "Addis Ababa", "Oromia", "Amhara", "Sidama", "Tigray", "Gambella", "Southern Ethiopia", "Southeastern Ethiopia", "Somali Region", 
        "Afar Zone 2", "Afar Zone 4", "Shabelle River", "Awash Region", "Fentale Volcano Area", "Awash-Metehara"
    ],
    "Kenya": [
        "Nairobi", "Kisumu", "Mombasa", "Kisumu Central", "Kisumu Nyando", "Muhoroni", "Nyakach", "Kisumu East", "Kisumu Kapuothe", 
        "Highlands East", "Rift Valley", "Coastal", "North-Western", "North-Eastern Kenya", "Busia", "Bunyala Sub-County", "Mombasa", 
        "Kapuothe Sub-County", "Dunga", "Highlands West of the Rift Valley", "Northern Rift Valley", "Southern Rift Valley", 
        "Highlands East of the Rift Valley"
    ],
    "Uganda": [
        "Kampala", "Mbale", "West Nile", "Jinja", "Kumi", "Ntoroko", "Namayingo", "Kikuube", "Pallisa", "Kagadi", "Butaleja", 
        "Kyenjojo", "Kaliro", "Bugiri", "Kibuku", "Namutumba", "Tororo", "Budaka", "Butebo", "Kisumu", "Kisumu Nyando", "Kisumu East", 
        "Kisumu Central", "Gulu", "Kamuli", "Bududa", "Eastern Uganda"
    ],
    "Somalia": [
        "Mogadishu", "Baidoa", "Beledweyne", "Hargeisa", "Afgoye", "Afgooye", "Lower Shabelle Region", "Middle Shabelle", "Hiiraan Region", 
        "Lower Juba", "Middle Juba", "Gedo", "Southern Somalia", "Somali Region (Ethiopia)", "Shabelle River"
    ],
    "Tanzania": [
        "Dar Es Salaam", "Arusha", "Mwanza", "Dodoma", "Zanzibar", "Serengeti", "Maasai Mara"
    ],
    "Burundi": [
        "Bujumbura", "Cibitoke", "Gatumba", "Mutimbuzi", "Mabayi", "Bubanza"
    ],
    "Rwanda": [
        "Kigali", "Rubavu", "Gatsata", "Jabana", "Western Rwanda", "Northern Rwanda"
    ]
    
}


In [53]:
#import pandas as pd

def get_adm_code_for_region(region, country, adm_level):
    try:
        geojson_data = get_adm(country, adm=adm_level, simplified=True)
        features = geojson_data['features']

        for feature in features:
            if region.lower() in feature['properties']['shapeName'].lower():
                return {
                    "shapeName": feature['properties']['shapeName'],
                    "shapeISO": feature['properties'].get('shapeISO'),
                    "shapeID": feature['properties'].get('shapeID'),
                    "shapeGroup": feature['properties'].get('shapeGroup'),
                    "shapeType": feature['properties'].get('shapeType')
                }
        return None
    except Exception as e:
        return {"Error": str(e)}

rows = []
adm_level = 1

for country, regions in regions_dict.items():
    for region in regions:
        adm_info = get_adm_code_for_region(region, country, adm_level)
        if adm_info:  
            rows.append({
                "Country": country,
                "Region": region,
                "shapeName": adm_info['shapeName'],
                "shapeISO": adm_info.get('shapeISO'),
                "shapeID": adm_info.get('shapeID'),
                "shapeGroup": adm_info.get('shapeGroup'),
                "shapeType": adm_info.get('shapeType')
            })

df = pd.DataFrame(rows)
output_path = "ea_gaul_geob_admin2_codeshare.csv"
df.to_csv(output_path, index=False)

print(f"CSV file created: {output_path}")


CSV file created: ea_gaul_geob_admin2_codeshare.csv
